### Simple RAG using Langchain and ChromaDB
- Query → Embed → Search Vector DB → Retrieve Docs → LLM (with context) → Answer

In [20]:
from langchain_community.document_loaders import WikipediaLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings


In [24]:
# Documents are loaded from Wikipedia
loader = WikipediaLoader(query="Transformer (deep learning)", load_max_docs=10)
documents = loader.load()
print(f" Loaded {len(documents)} Wikipedia pages")
print(f"\nFirst document preview:")
print(documents[0].page_content[:200] + "...")

/Users/yashsarode/Downloads/Personal Projects/Python/LangGraph-personal/.venv/lib/python3.13/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/yashsarode/Downloads/Personal Projects/Python/LangGraph-personal/.venv/lib/python3.13/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


 Loaded 9 Wikipedia pages

First document preview:
In deep learning, the transformer is an artificial neural network architecture based on the multi-head attention mechanism, in which text is converted to numerical representations called tokens, and e...


In [25]:
# Document is split into chunks of 1000 characters, with 200 characters overlap
# Why chunk?
        # - Embeddings work better on focused text
        # - Retrieval is more precise
        # - Fits in LLM context window
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)
print(f" Split into {len(docs)} chunks")
print(f"\nFirst chunk preview:")
print(docs[0].page_content[:200] + "...")


 Split into 47 chunks

First chunk preview:
In deep learning, the transformer is an artificial neural network architecture based on the multi-head attention mechanism, in which text is converted to numerical representations called tokens, and e...


In [26]:
# Convert documents to embeddings
# Why HuggingFace embeddings?
#         - Free, no API calls
#         - Runs locally (privacy + speed)
#         - Good quality for most use cases
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={"device": "cpu"},
    encode_kwargs={'normalize_embeddings': True}
)

# Embed the documents
vector_1 = embeddings.embed_query(docs[0].page_content)
vector_2 = embeddings.embed_query(docs[1].page_content)
print(vector_1[:10])

[-0.010038163512945175, -0.010613719001412392, -0.007397179026156664, 0.034642405807971954, -0.04844551533460617, 0.0017753464635461569, 0.03124556504189968, -0.0011594544630497694, -0.042442891746759415, -0.05231810733675957]


In [22]:
# Create a new vector store from documents.
        
# Process:
# 1. Each document is embedded (text → vector)
# 2. Vectors are indexed for fast search
# 3. Persisted to disk for reuse
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="simple_rag",
    embedding_function=embeddings,
    persist_directory="./chroma_db",  # Where to save data locally, remove if not necessary
)

In [20]:
vector_store.add_documents(docs)

['9ed268d3-6e62-4bfb-a94d-394553689db6',
 '74bf96e5-2699-4b89-b3d1-a34a68d19337',
 'a6785169-4cf0-4d68-9e2f-425c0192fae5',
 'ec94d334-b8f9-40f2-86df-03379f9f7fa2',
 'd36d1dc2-eb15-4fc0-9cb4-53a871e22638',
 '4434912e-5efd-4062-9f17-1c179abaff88',
 '1f071331-50ed-461a-ac3d-310c71f46d36',
 'c56a069c-c581-4f30-81d9-45e40f9e30bf',
 'bee45508-7b73-41be-9542-0741201193b3',
 '40a0a195-4440-408b-8147-f83fec19b6ba',
 '347b2dde-3bbd-43b2-bf43-24cf311b719e',
 '1589a3e3-a0b9-4cd3-9333-45fd33acb745',
 'f1f8bb3b-5add-4eb3-a9ab-838e0e9d3392',
 '22bf9e8e-b069-4bab-9b98-231235b931fa',
 '05b49edb-01e2-4ba6-a7bf-9736998f4ff3',
 '7e62f641-2e97-49bb-8789-79f0b80042b9',
 'de73c7da-3a57-431c-b35d-8be115db9978',
 '8650ff99-85a4-4bbf-a165-213a74e53da5',
 '4b2cc08b-e87e-432e-a127-46c6d6f1043f',
 'caf14c86-79dd-4a7f-b81e-df6f2fbcd241',
 '449282dc-bc01-4964-a216-35655612c1a1',
 '8cb1df8c-daba-4f2d-8e65-7b7d048ea8d2',
 '7c784813-e456-4e0c-8c48-dc31dd7af803',
 '0272c611-4581-4ce6-b125-176e4a46c0d9',
 'ba5be133-eacc-

In [26]:
query = "What advantage does the transformer model have over the RNN/LSTM model?"
results = vector_store.similarity_search(
    query, k=4
)

print(results)

[Document(id='74bf96e5-2699-4b89-b3d1-a34a68d19337', metadata={'source': 'https://en.wikipedia.org/wiki/Transformer_(deep_learning)', 'title': 'Transformer (deep learning)', 'summary': 'In deep learning, the transformer is an artificial neural network architecture based on the multi-head attention mechanism, in which text is converted to numerical representations called tokens, and each token is converted into a vector via lookup from a word embedding table. At each layer, each token is then contextualized within the scope of the context window with other (unmasked) tokens via a parallel multi-head attention mechanism, allowing the signal for key tokens to be amplified and less important tokens to be diminished. \nTransformers have the advantage of having no recurrent units, therefore requiring less training time than earlier recurrent neural architectures (RNNs) such as long short-term memory (LSTM). Later variations have been widely adopted for training large language models (LLMs) o

In [27]:
print(f"Query: {query}")
print(f"Retrieved {len(results)} documents")

for i,r in enumerate(results):
    print(f"Document {i+1}:")
    print(r.page_content[:200] + "...")
    print("\n")

Query: What advantage does the transformer model have over the RNN/LSTM model?
Retrieved 4 documents
Document 1:
The modern version of the transformer was proposed in the 2017 paper "Attention Is All You Need" by researchers at Google. The predecessors of transformers were developed as an improvement over previo...


Document 2:
Modern transformers overcome this problem, but unlike RNNs, they require computation time that is quadratic in the size of the context window. The linearly scaling fast weight controller (1992) learns...


Document 3:
In deep learning, the transformer is an artificial neural network architecture based on the multi-head attention mechanism, in which text is converted to numerical representations called tokens, and e...


Document 4:
The transformer architecture for deep learning is the core technology of a GPT. Developed by researchers at Google, it was introduced in the paper "Attention Is All You Need", which was published on J...




In [29]:
from langchain_core.documents.base import Document


results= vector_store.similarity_search_with_score(query, k=4)
        
print(f"\n📊 Search results with scores:")
for i, (doc, score) in enumerate[tuple[Document, float]](results, 1):
    print(f"{i}. Score: {score:.4f} | Source: {doc.metadata.get('source', 'Unknown')}")
    print(f"   Preview: {doc.page_content[:100]}...\n")


📊 Search results with scores:
1. Score: 0.7498 | Source: https://en.wikipedia.org/wiki/Transformer_(deep_learning)
   Preview: The modern version of the transformer was proposed in the 2017 paper "Attention Is All You Need" by ...

2. Score: 0.9087 | Source: https://en.wikipedia.org/wiki/Transformer_(deep_learning)
   Preview: Modern transformers overcome this problem, but unlike RNNs, they require computation time that is qu...

3. Score: 0.9444 | Source: https://en.wikipedia.org/wiki/Transformer_(deep_learning)
   Preview: In deep learning, the transformer is an artificial neural network architecture based on the multi-he...

4. Score: 0.9529 | Source: https://en.wikipedia.org/wiki/Generative_pre-trained_transformer
   Preview: The transformer architecture for deep learning is the core technology of a GPT. Developed by researc...



### Runnable: @chain vs as_retriever()
- Runnable: a standard abstraction that represents a single unit of work (transforming input → output), with support for sync, async, batching, streaming, tracing/debugging, composition, etc.
- The @chain decorator converts a normal Python function into a Runnable
- @chain gives you a “chainable, runnable function” — you can treat it as part of a larger workflow, or call it on inputs (single or batch), just like built-in Runnables (retrievers, LLM wrappers, etc.).

In [ ]:
# a retriever interface for use in chains.
# Retriever is LangChain's abstraction for "anything that retrieves documents".
# This makes your RAG chain modular - you can swap retrievers easily.
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain

@chain
def retrieverFn(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=3)


retrieverFn.batch(
    [
        query,
        "What is Parallelizing attention?",
    ],
)



[[Document(id='74bf96e5-2699-4b89-b3d1-a34a68d19337', metadata={'source': 'https://en.wikipedia.org/wiki/Transformer_(deep_learning)', 'title': 'Transformer (deep learning)', 'summary': 'In deep learning, the transformer is an artificial neural network architecture based on the multi-head attention mechanism, in which text is converted to numerical representations called tokens, and each token is converted into a vector via lookup from a word embedding table. At each layer, each token is then contextualized within the scope of the context window with other (unmasked) tokens via a parallel multi-head attention mechanism, allowing the signal for key tokens to be amplified and less important tokens to be diminished. \nTransformers have the advantage of having no recurrent units, therefore requiring less training time than earlier recurrent neural architectures (RNNs) such as long short-term memory (LSTM). Later variations have been widely adopted for training large language models (LLMs) 

### What is as_retriever() ?
- as_retriever() is a convenience method on a vector store that returns a Retriever object (specifically a VectorStoreRetriever) — a class specialized for retrieval tasks.
- as_retriever() is about creating a retriever adapter over your vector store, exposing standardized methods to retrieve documents given a query, optionally with retrieval params.

### How do they work together?
- as_retriever() deals with what you want to do: “retrieve relevant docs from vector store.”
- @chain deals with how you integrate logic (possibly including the retriever) into a larger, composable, traceable pipeline.
- A common pattern is to call a retriever (via .get_relevant_documents() or .similarity_search()) inside a function decorated with @chain — exactly like your snippet. Then you can pipe that custom chain further (e.g. pass retrieved docs into prompt → LLM → parser), and everything remains a Runnable.

In [36]:
retriever = vector_store.as_retriever(
    search_type="similarity", search_kwargs={"k": 3}
)
search_docs = retriever.batch([
        query,
        "What is Parallelizing attention?",
    ], filter={"title": "Transformer (deep learning)"})

print(len(search_docs))
search_docs[0]
search_docs[1]

2


[Document(id='9ed268d3-6e62-4bfb-a94d-394553689db6', metadata={'title': 'Transformer (deep learning)', 'summary': 'In deep learning, the transformer is an artificial neural network architecture based on the multi-head attention mechanism, in which text is converted to numerical representations called tokens, and each token is converted into a vector via lookup from a word embedding table. At each layer, each token is then contextualized within the scope of the context window with other (unmasked) tokens via a parallel multi-head attention mechanism, allowing the signal for key tokens to be amplified and less important tokens to be diminished. \nTransformers have the advantage of having no recurrent units, therefore requiring less training time than earlier recurrent neural architectures (RNNs) such as long short-term memory (LSTM). Later variations have been widely adopted for training large language models (LLMs) on large (language) datasets.\n\nThe modern version of the transformer w

### Create a tool that makes a vector search 
This tool will be given to an LLM via an agent. i.e create_agent

In [15]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=3)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

### Load and initialise LLM to be used in the agent

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [5]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
)

In [6]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="J'adore la programmation.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 55, 'total_tokens': 64, 'completion_time': 0.027728082, 'completion_tokens_details': None, 'prompt_time': 0.001947106, 'prompt_tokens_details': None, 'queue_time': 0.058577383, 'total_time': 0.029675188}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--979d3217-86c6-44fd-bb61-542c0a2fb6ff-0', usage_metadata={'input_tokens': 55, 'output_tokens': 9, 'total_tokens': 64})

### Define agent with tools and system prompt

In [28]:
from langchain.agents import create_agent


tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
     "You have access to a tool that retrieves context from a blog post. "
    "Use the tool to help answer user queries."
)
agent = create_agent(llm, tools, system_prompt=prompt)

###  This agentic RAG formulation we allow the LLM to use its discretion in generating a tool call to help answer user queries.
Drawbacks:
- Two inference calls – When a search is performed, it requires one call to generate the query and another to produce the final response.
- Reduced control – The LLM may skip searches when they are actually needed, or issue extra searches when unnecessary.

In [ ]:
query = (
    "What was Elman network example and what year it was in? ?\n\n"
    "Once you get that answer, how it affected the RNNs."
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What was Elman network example and what year it was in? ?

Once you get that answer, how it affected the RNNs.
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (rwwwewjac)
 Call ID: rwwwewjac
  Args:
    query: Elman network example and year
  retrieve_context (sh74g5px5)
 Call ID: sh74g5px5
  Args:
    query: Elman network impact on RNNs
================================= Tool Message =================================
Name: retrieve_context

Source: {'title': 'Attention (machine learning)', 'source': 'https://en.wikipedia.org/wiki/Attention_(machine_learning)', 'summary': 'In machine learning, attention is a method that determines the importance of each component in a sequence relative to the other components in that sequence. In natural language processing, importance is represented by "soft" weights assigned to each word in a sentence. More 

### 2 step RAG chain that uses single LLM call per query.
- @dynamic_prompt you are defining a function that, at runtime, builds a system prompt string based on the current request and agent state.

- Why is it in middleware and used with create_agent?
-> Middleware gives control over the agent lifecycle.
The middleware abstraction in LangChain lets you hook into the agent’s lifecycle: before model call, after model call, wrap model call, etc.

In [33]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vector_store.similarity_search(last_query)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    system_message = (
        "You are a helpful assistant. Use the following context in your response:"
        f"\n\n{docs_content}"
    )

    return system_message


agent = create_agent(llm, tools=[], middleware=[prompt_with_context])
query = (
    "What was Elman network example and what year it was in? ?\n\n"
    "Once you get that answer, how it affected the RNNs."
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What was Elman network example and what year it was in? ?

Once you get that answer, how it affected the RNNs.
================================== Ai Message ==================================

The Elman network example was a well-cited early example of sequence modelling and generation using plain recurrent neural networks (RNNs), and it was in the year 1990.

The Elman network affected RNNs in that it demonstrated the potential of RNNs for sequence modelling, but it also highlighted the limitations of early RNNs. Specifically, the Elman network suffered from the vanishing-gradient problem, which made it difficult for the model to capture long-range dependencies in sequences. This meant that the information from one token could propagate arbitrarily far down the sequence in theory, but in practice, the model's state at the end of a long sentence would not have precise, extractable information about preced